In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

In [2]:
dataurl = 'https://docs.google.com/spreadsheets/d/1EmeVCJzCiMISggPuS5clv-gz4mznq24NMyFV8aXkbK4/edit?gid=0#gid=0'
dataurl = dataurl.replace('/edit?gid=', '/export?format=csv&gid=')
dataset = pd.read_csv(dataurl)
dataset = dataset.fillna('')
refDataset = dataset
dataset

,My Gender,My age,My city of residence,Video Introductions\n\n\n\nSubmit this if you wish to share yours https://forms.gle/bvNGD2CcixYQV7sYA,"I define privilege to be (having a supportive family, dependable friends, a job you love, anything else) the more detailed your response the more likely you will be asked for an introduction.","About my growing up years (they were awesome, they were challenging but all is good now, about the family, about friends, memories etc etc) the more detailed your response the more likely you will be asked for an introduction.",My current relationship status,I have children,Ball park of my professional annual income,"How do I introduce the professional me (banker, IT professional, entrepreneur, environmentalist) the more detailed your response the more likely you will be asked for an introduction.",...,Educational degree,Anything else that you may like to mention to the community members (any specifics which may be important to you) the more detailed your response the more likely you will be asked for an introduction.,"Submitting this document means, I have shared factual information + giving consent (barring your personal details - full name and contact number) for your details to be viewed by andwemet singles community members.",My religious inclination,I am open to meeting someone no more married,I am fine with meeting a single parent,Open to meeting someone form faith not similar to mine,How did you hear about the community,Look for your row number in the excel sheet once you have submitted this doc. Your row# on submitting this doc will be found here,Video Introduction
0,Male,32,Mumbai,,Having things that money cant buy.,"Defense upbringing, kept moving cities and sch...","Separated, filed for a divorce",No,Between 40-50 Lakhs,Finance Professional,...,Masters,Health concious due to health conditions in fa...,I give consent,,,,,,,
1,Male,36,New Delhi,,"Family,friends",Awesome,Never married,No,Between 30-35 lakhs,Education institute owner,...,Masters,I like playing sports,I give consent,,,,,,,
2,Male,32,Bangalore,,Supportive/understandable/Friendly,"Awesome & challenging, All Good Now.",Never married,No,Between 12-15 Lakhs,Engineer (Interior Design),...,Masters,No,I give consent,,,,,,,
3,Female,33,Gurugram,,Being able to love people and be loved by them,We’re full of curiosity and learning I was an ...,Was married earlier,No,Between 18-20 Lakhs,Mental health professional,...,Phd and equivalent,I take time to open up and quite curious,I give consent,,,,,,,
4,Female,32,Mumbai,,Being able to choose and make a life for myself,"Coming from a humble background, it has taught...",Never married,No,Between 18-20 Lakhs,Producer in Advertising,...,PG Diploma,I believe honest communication and mutual resp...,I give consent,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,Female,37,Bangalore,,Having someone who care for you in any given s...,It was extremely tough and challenging but it ...,Was married earlier,No,Between 15-18 Lakhs,IT professional,...,Undergraduate,I prefer Simplicity and Honesty,I give consent,Christian,I am not sure,I am not sure,Yes,Instagram,,
324,Male,40,Noida,,Have a supportive family and friends,It was good and challenging both,"Separated, filed for a divorce",No,Between 18-20 Lakhs,IT Professional,...,Masters,I love dogs and wanted to keep that in future....,I give consent,Hindu,Yes,Yes I am,Yes,Instagram,,
325,Female,48,Pune,,Just to be breathing every day and having a pe...,It was mix of happy and not so happy memories.,Was married earlier,Yes,Between 12-15 Lakhs,Solopreneur and Coach,...,PG Diploma,I am divorced and have a 17 year old daughter.,I give consent,Hindu,Yes,Yes I am,I am not sure,Other,,
326,Male,49,Pune,,Human Life and our ability to make it beautiful!,I am lucky to have a lot many life experiences...,Never married,No,Over 90 Lakhs,"Held global positions in HR, Sales Ops, Merger...",...,Masters,I believe in : Just li

In [3]:
dataset.index = np.arange(2, len(dataset) + 2)
dataset

,My Gender,My age,My city of residence,Video Introductions\n\n\n\nSubmit this if you wish to share yours https://forms.gle/bvNGD2CcixYQV7sYA,"I define privilege to be (having a supportive family, dependable friends, a job you love, anything else) the more detailed your response the more likely you will be asked for an introduction.","About my growing up years (they were awesome, they were challenging but all is good now, about the family, about friends, memories etc etc) the more detailed your response the more likely you will be asked for an introduction.",My current relationship status,I have children,Ball park of my professional annual income,"How do I introduce the professional me (banker, IT professional, entrepreneur, environmentalist) the more detailed your response the more likely you will be asked for an introduction.",...,Educational degree,Anything else that you may like to mention to the community members (any specifics which may be important to you) the more detailed your response the more likely you will be asked for an introduction.,"Submitting this document means, I have shared factual information + giving consent (barring your personal details - full name and contact number) for your details to be viewed by andwemet singles community members.",My religious inclination,I am open to meeting someone no more married,I am fine with meeting a single parent,Open to meeting someone form faith not similar to mine,How did you hear about the community,Look for your row number in the excel sheet once you have submitted this doc. Your row# on submitting this doc will be found here,Video Introduction
2,Male,32,Mumbai,,Having things that money cant buy.,"Defense upbringing, kept moving cities and sch...","Separated, filed for a divorce",No,Between 40-50 Lakhs,Finance Professional,...,Masters,Health concious due to health conditions in fa...,I give consent,,,,,,,
3,Male,36,New Delhi,,"Family,friends",Awesome,Never married,No,Between 30-35 lakhs,Education institute owner,...,Masters,I like playing sports,I give consent,,,,,,,
4,Male,32,Bangalore,,Supportive/understandable/Friendly,"Awesome & challenging, All Good Now.",Never married,No,Between 12-15 Lakhs,Engineer (Interior Design),...,Masters,No,I give consent,,,,,,,
5,Female,33,Gurugram,,Being able to love people and be loved by them,We’re full of curiosity and learning I was an ...,Was married earlier,No,Between 18-20 Lakhs,Mental health professional,...,Phd and equivalent,I take time to open up and quite curious,I give consent,,,,,,,
6,Female,32,Mumbai,,Being able to choose and make a life for myself,"Coming from a humble background, it has taught...",Never married,No,Between 18-20 Lakhs,Producer in Advertising,...,PG Diploma,I believe honest communication and mutual resp...,I give consent,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,Female,37,Bangalore,,Having someone who care for you in any given s...,It was extremely tough and challenging but it ...,Was married earlier,No,Between 15-18 Lakhs,IT professional,...,Undergraduate,I prefer Simplicity and Honesty,I give consent,Christian,I am not sure,I am not sure,Yes,Instagram,,
326,Male,40,Noida,,Have a supportive family and friends,It was good and challenging both,"Separated, filed for a divorce",No,Between 18-20 Lakhs,IT Professional,...,Masters,I love dogs and wanted to keep that in future....,I give consent,Hindu,Yes,Yes I am,Yes,Instagram,,
327,Female,48,Pune,,Just to be breathing every day and having a pe...,It was mix of happy and not so happy memories.,Was married earlier,Yes,Between 12-15 Lakhs,Solopreneur and Coach,...,PG Diploma,I am divorced and have a 17 year old daughter.,I give consent,Hindu,Yes,Yes I am,I am not sure,Other,,
328,Male,49,Pune,,Human Life and our ability to make it beautiful!,I am lucky to have a lot many life experiences...,Never married,No,Over 90 Lakhs,"Held global positions in HR, Sales Ops, Merger...",...,Masters,I believe in : Just li

In [4]:
dataset = (dataset.iloc[:,:3]).join(dataset.iloc[:,6])

In [5]:
dataset

,My Gender,My age,My city of residence,My current relationship status
2,Male,32,Mumbai,"Separated, filed for a divorce"
3,Male,36,New Delhi,Never married
4,Male,32,Bangalore,Never married
5,Female,33,Gurugram,Was married earlier
6,Female,32,Mumbai,Never married
...,...,...,...,...
325,Female,37,Bangalore,Was married earlier
326,Male,40,Noida,"Separated, filed for a divorce"
327,Female,48,Pune,Was married earlier
328,Male,49,Pune,Never married


In [6]:
age_weight =2.0
relationship_weight = 1.5
city_weight = 1.0


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
city_vectorized = vectorizer.fit_transform(dataset['My city of residence'])

In [8]:
# one hot encoding
encoded_categorical_features =  (pd.get_dummies(dataset['My current relationship status'], dtype=int))

In [9]:
encoded_categorical_features

,Never married,"Separated, filed for a divorce","Separated, not legally divorced",Was married earlier,Widow/ Widower
2,0,1,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
5,0,0,0,1,0
6,1,0,0,0,0
...,...,...,...,...,...
325,0,0,0,1,0
326,0,1,0,0,0
327,0,0,0,1,0
328,1,0,0,0,0


In [10]:
#label encode gender

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
labeledFeatures = labelencoder.fit_transform(dataset['My Gender'])

In [11]:
dataset

,My Gender,My age,My city of residence,My current relationship status
2,Male,32,Mumbai,"Separated, filed for a divorce"
3,Male,36,New Delhi,Never married
4,Male,32,Bangalore,Never married
5,Female,33,Gurugram,Was married earlier
6,Female,32,Mumbai,Never married
...,...,...,...,...
325,Female,37,Bangalore,Was married earlier
326,Male,40,Noida,"Separated, filed for a divorce"
327,Female,48,Pune,Was married earlier
328,Male,49,Pune,Never married


In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_numerical_features= scaler.fit_transform((dataset.iloc[:,1].values).reshape(-1,1))

In [13]:
dataset

,My Gender,My age,My city of residence,My current relationship status
2,Male,32,Mumbai,"Separated, filed for a divorce"
3,Male,36,New Delhi,Never married
4,Male,32,Bangalore,Never married
5,Female,33,Gurugram,Was married earlier
6,Female,32,Mumbai,Never married
...,...,...,...,...
325,Female,37,Bangalore,Was married earlier
326,Male,40,Noida,"Separated, filed for a divorce"
327,Female,48,Pune,Was married earlier
328,Male,49,Pune,Never married


In [14]:
labeledFeatures=labeledFeatures.reshape(-1,1)

print(city_vectorized.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [15]:
# prepare the data for training
features = np.hstack([labeledFeatures, scaled_numerical_features, encoded_categorical_features.values, city_vectorized.toarray()])


In [16]:
print(features.shape)

(328, 139)


In [17]:
k =280
knn = NearestNeighbors(n_neighbors=k, metric='cosine')
knn.fit(features)

NearestNeighbors(metric='cosine', n_neighbors=280)

In [18]:
row =210
selectedRow = refDataset.iloc[row]

In [19]:
distances, indices = knn.kneighbors(features[row].reshape(1,-1))
print(indices)

[[210 220 113 109 252  11 316 108 272 165  45 266 313  25 320  41 278 326
  245 247 156 290  48 261  24 311  81 229  26 139 309 181 256 288 123  51
   54 118  97 122  30 110 295 128 262 136  84  86  19 231 260 325  55 259
  274  75 310 307 114 183  42 287 265 200 308  52  32 100  18 105  50  73
    1  27 174 327 185   6  29 258 111  12  65 251 228 301 178 282 291 246
  221 166 296 121  96  66  77  56 244  83 201  36 196 169 135 237 106 209
  160  91 241  89 143 194 163  71 171 208 219 255 321 107 230 184 238  21
  112 264 271 104 314 119 173 214 211  35 205 142  53 263 147 249  62  76
  141 242  70  94 191 137  79  59  37 180 315 188 257 101   9 297 152  23
  324  61 179 276   8  57 131 117 293  60 126  43 279 250  13  88 130 223
   33 199  22 217  10 236 189 133  46  14 132 146 170 195 319 212 234 216
  124 134 299 213  95 269 127 253 154 273 305 292 192 145  16  72 285 153
   85 233 232  40  49 138 222 294  39  38 225  44 103 140 284 167 203 129
  190 182  69   2 317   4 267 102 204 

In [20]:
for i in indices[0]:
    if(dataset.iloc[i]['My Gender'] == selectedRow["My Gender"]):
        continue
    print(dataset.iloc[i].values)
    print("")
    print("")


['Male' 43 'Delhi' 'Never married']


['Male' 55 'Dubai ' 'Never married']


['Male' 54 'Delhi ' 'Was married earlier']


['Male' 49 'Pune' 'Never married']


['Male' 45 'Delhi ' 'Was married earlier']


['Male' 45 'Mumbai ' 'Never married']


['Male' 37 'Delhi' 'Never married']


['Male' 44 'Delhi' 'Was married earlier']


['Male' 49 'New Delhi ' 'Was married earlier']


['Male' 43 'Delhi' 'Was married earlier']


['Male' 36 'Delhi' 'Never married']


['Male' 36 'Delhi ' 'Never married']


['Male' 42 'Mumbai' 'Never married']


['Male' 42 'Bangalore ' 'Never married']


['Male' 38 'New Delhi' 'Never married']


['Male' 38 'New Delhi' 'Never married']


['Male' 45 'New Delhi ' 'Was married earlier']


['Male' 50 'Bangalore' 'Widow/ Widower']


['Male' 35 'Delhi' 'Never married']


['Male' 40 'delhi' 'Was married earlier']


['Male' 40 'Bangalore ' 'Never married']


['Male' 40 'Hyderabad/pune' 'Never married']


['Male' 48 'Bangalore' 'Separated, not legally divorced']


['Male' 48 'Mu

In [21]:
def preprocessData(dataset):
    dataset = (dataset.iloc[:,:3]).join(dataset.iloc[:,6])

    vectorizer = TfidfVectorizer()
    city_vectorized = vectorizer.fit_transform(dataset['My city of residence'])

    # one hot encoding
    encoded_categorical_features =  (pd.get_dummies(dataset['My current relationship status'], dtype=int))

    #label encode
    labelencoder = LabelEncoder()
    labeledFeatures = labelencoder.fit_transform(dataset['My Gender'])

    scaler = StandardScaler()
    scaled_numerical_features= scaler.fit_transform((dataset.iloc[:,1].values).reshape(-1,1))

    labeledFeatures=labeledFeatures.reshape(-1,1)
    city_vectorized = city_vectorized.toarray()

    features = np.hstack([labeledFeatures, scaled_numerical_features, encoded_categorical_features.values, city_vectorized])
    return features

In [22]:
print(preprocessData(refDataset))

[[ 1.         -0.88707346  0.         ...  0.          0.
   0.        ]
 [ 1.         -0.20963436  1.         ...  0.          0.
   0.        ]
 [ 1.         -0.88707346  1.         ...  0.          0.
   0.        ]
 ...
 [ 0.          1.82268295  0.         ...  0.          0.
   0.        ]
 [ 1.          1.99204272  1.         ...  0.          0.
   0.        ]
 [ 1.          1.4839634   0.         ...  0.          0.
   0.        ]]


In [23]:
joblib.dump(knn, 'simpleclassifier.joblib')

['simpleclassifier.joblib']